In [5]:
##Run these in the console if you don't already have them installed <3
# pip install -U pip setuptools wheel #to get pip
# pip install -U spacy #to get spacy
# pip install lemminflect #to get lemmainflect -> eng only (even pyinflect recommends using lemminflect instead)
# python -m spacy download en_core_web_md ##md size is minimum that has vectors for words
# python -m spacy download pl_core_news_md

ModuleNotFoundError: No module named 'spacy'